In [87]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Extraindo os dados

In [113]:
#Criando um dicionário com todos os caminhos dos arquivos. Isso vai ser bom porque utilizarei as chaves para criar uma coluna de "Ano"
arquivos = {
    '2022': 'data/dados_vacinados_2022.csv',
    '2023': 'data/dados_vacinados_2023.csv',
    '2024': 'data/dados_vacinados_2024.csv'
}

lista_dataframes = [] #Para armazenar todos os dataframes

for ano, arquivo in arquivos.items():
    try:
        df = pd.read_csv(arquivo, encoding='utf-8') #No meu caso, eu não utilizei "sep=';'" porque esse dataset é separado por vígula mesmo, que é o padrão da função

        df['ano_origem'] = ano #Utilizando o a chave "ano" para criar uma coluna nova em cada dataframe com o ano correspondente

        lista_dataframes.append(df)
    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")


df = pd.concat(lista_dataframes, ignore_index=True) #Concatenando todos os DFs

<h2> Transformação </h2>

In [114]:
#Verificando o resultado final e as informações sobre os tipos de dados e 
display(df.head(5))
display(df.info())

,_id,faixa_etaria,idade,sexo,raca_cor,municipio,grupo,categoria,lote,vacina_fabricante,descricao_dose,cnes,sistema_origem,data_vacinacao,ano_origem
0,1,35 a 39 anos,37.0,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),OUTRAS,FM2948,3 - COMIRNATY (PFIZER),2,DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER,Conecta Recife,2022-03-25T00:00:00,2022
1,2,50 a 54 anos,52.0,MASCULINO,BRANCA,RECIFE,PÚBLICO EM GERAL (18 a 59 anos),OUTROS,FT5177,3 - COMIRNATY (PFIZER),4,DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER,Conecta Recife,2022-12-06T00:00:00,2022
2,3,55 a 59 anos,58.0,FEMININO,BRANCA,OLINDA,PÚBLICO EM GERAL (18 a 59 anos),OUTROS,FP7082,3 - COMIRNATY (PFIZER),1,DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BA...,Conecta Recife,2022-06-04T00:00:00,2022
3,4,35 a 39 anos,35.0,MASCULINO,PARDA,RECIFE,TRABALHADORES DA EDUCAÇÃO,REDE PRIVADA - ENSINO FUNDAMENTAL,205H21A,4 - JANSSEN COVID-19 VACCINE (JOHNSON & JOHNSON),3,DS 8: CNES: 0001198 - UBT ARISTARCHO DE AZEVEDO,Conecta Recife,2022-03-03T00:00:00,2022
4,5,40 a 44 anos,40.0,MASCULINO,PRETA,RECIFE,TRABALHADORES DA SAÚDE,NaN,219VCD273Z,2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ),2,DS 6: CNES: 0001392 - MIGUEL DE LIMA VALVERDE.,Conecta Recife,2022-02-08T00:00:00,2022


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406500 entries, 0 to 406499
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   _id                406500 non-null  int64  
 1   faixa_etaria       406498 non-null  object 
 2   idade              406499 non-null  float64
 3   sexo               406493 non-null  object 
 4   raca_cor           406426 non-null  object 
 5   municipio          406498 non-null  object 
 6   grupo              406500 non-null  object 
 7   categoria          50010 non-null   object 
 8   lote               406413 non-null  object 
 9   vacina_fabricante  406500 non-null  object 
 10  descricao_dose     406500 non-null  int64  
 11  cnes               406500 non-null  object 
 12  sistema_origem     406500 non-null  object 
 13  data_vacinacao     406500 non-null  object 
 14  ano_origem         406500 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 4

None

<h3> Verificando valores únicos em cada uma das minhas colunas. </h3>

In [115]:
df.nunique(dropna=False) #Usar o "dropna=False" faz com que ele leve em consideração valores nunlos

_id                  379750
faixa_etaria             22
idade                   114
sexo                      4
raca_cor                  8
municipio               312
grupo                    27
categoria               156
lote                    315
vacina_fabricante         6
descricao_dose            5
cnes                      9
sistema_origem            2
data_vacinacao          361
ano_origem                3
dtype: int64

<h3> Verificando valores duplicados e removendo </h3>

In [116]:
df_duplicados = df.copy()
df_duplicados = df_duplicados.drop('_id', axis=1)
print("Linhas duplicadas: ", df_duplicados.duplicated().sum())

Linhas duplicadas:  114878


<p>Ou, você pode fazer dessa forma usando a função "subset" do "duplicated"</p>

In [117]:
print(f"Linhas ANTES de remover duplicatas: {len(df)}")

todas_colunas = df.columns.tolist()

colunas_para_verificar = [coluna for coluna in todas_colunas if coluna != "_id"]

total_duplicatas = df.duplicated(subset=colunas_para_verificar, keep='first').sum()

print(f"Total de linhas duplicadas: {total_duplicatas}")

df = df.drop_duplicates(subset=colunas_para_verificar, keep='first')

print(f"Linhas DEPOIS de remover duplicatas: {len(df)}")

Linhas ANTES de remover duplicatas: 406500
Total de linhas duplicadas: 114878
Linhas DEPOIS de remover duplicatas: 291622


In [118]:
df = df.drop_duplicates()

<h3>Verificando os valores nulos por coluna</h3>

In [119]:
print("Valores ausentes por coluna:")
print(df.isnull().sum())

Valores ausentes por coluna:
_id                       0
faixa_etaria              2
idade                     1
sexo                      7
raca_cor                 74
municipio                 2
grupo                     0
categoria            243486
lote                     83
vacina_fabricante         0
descricao_dose            0
cnes                      0
sistema_origem            0
data_vacinacao            0
ano_origem                0
dtype: int64


<p>Verificando a quantidade de valores nulos, iremos começar a tratar alguns deles.</p>
<ul>
    <li>sexo: Apenas 7 linhas vazias. Nessa coluna, temos os valores "MACULINO", "FEMININO" e "OUTROs". Podemos passar todas as linhas nulas para "OUTROS"</li>
    <li>raca_cor: Temos 74 linhas com valores nulos. Nessa coluna, temos os valores 'PARDA', 'BRANCA', 'PRETA', 'AMARELA', 'INDÍGENA', 'ÍNDIGENA' e
       'NÃO INFORMADO'. Para todos os nulos, podemos apenas colocar "NÃO INFORMADO".</li>
    <li>categoria: Temos 155 valores diferentes (156 se contamor os nulos). Entre esses valores, temos "OUTROS" e "OUTRAS". Primeiro, podemos padronizar e tornar todos como apenas "OUTROS", e, posteriormente, classifica-los todos os nulos como "OUTROS". </li>
    <li>lote: Temos 87 linhas com valores nulos. Poderíamos criar uma nova categoria de "Não informado" e popular essas linhas com essa nova categoria. Entretanto, particularmente, eu acho que essa coluna não agrega muito para o dataset, então, iremos excluí-la posteriormente.</li>
</ul>
<p></p>

In [120]:
#Vamos printar os valores únicos de algumas colunas
print("Valores únicos da coluna 'sexo': \n", df['sexo'].unique())
print("Valores únicos da coluna 'raca_cor': \n", df['raca_cor'].unique())
print("Valores unicos da coluna 'categoria' (Top 10): \n", df['categoria'].value_counts().head(10))
print("Valores unicos da coluna 'lote' (Top 10): \n", df['lote'].value_counts().head(10))

Valores únicos da coluna 'sexo': 
 ['MASCULINO' 'FEMININO' 'OUTROS' nan]
Valores únicos da coluna 'raca_cor': 
 ['PARDA' 'BRANCA' 'PRETA' 'AMARELA' 'INDÍGENA' 'ÍNDIGENA' nan
 'NÃO INFORMADO']
Valores unicos da coluna 'categoria' (Top 10): 
 categoria
OUTRAS                                                                                            9423
DIABETES MELLITUS                                                                                 4752
OUTROS                                                                                            3929
GRÁVIDAS                                                                                          2610
HIPERTENSÃO ARTERIAL RESISTENTE NOS ESTÁGIOS 1,2 E 3 COM LESÃO EM ÓRGÃO-ALVO E/OU COMORBIDADE.    2131
PESSOAS COM OBESIDADE MÓRBIDA                                                                     1592
CARDIOPATIA HIPERTENSIVA                                                                          1334
IMUNOSSUPRIMIDOS            

<p>Agora, vamos tratar preencher os valores nulos com os valores que definimos.</p>

In [121]:
#Aplicando as transformações:

df['sexo'] =  df['sexo'].fillna('OUTROS') #Preenchendo os valores nulos da coluna 'sexo' por "OUTROS"
df['raca_cor'] = df['raca_cor'].fillna('NÃO INFORMADO') #Preenchendo os valores nulos da coluna 'raca_cor' para "NÃO INFORMADO"
df.loc[df['categoria'] == "OUTRAS", "categoria"] = "OUTROS" #Primeiro, substituindo os valores "OUTRAS" para "OUTROS", para padronizar
df['categoria'] = df['categoria'].fillna('OUTROS') #Agora, preenchendo os valores nulos com "OUTROS"

<h3>Padronizando colunas</h3>

<p>Agora, iremos padronizar algumas colunas.</p>
<p>A primeira coluna será a de "grupos". Essa coluna possui diversos valores que podem ser simplificados. Por exemplo, eixstem vários valores diferentes para crianças, como "CRIANÇAS DE 03 E 04 ANOS" e "CRIANÇAS DE 05 A 11 ANOS - Prioritárias", que no fim, podem ser simplificadas para apenas "Crianças". Isso facilitará caso a gente queira realizar alguma análise nesses grupos posteriormente.</p>>

In [122]:
print("\nValores únicos na coluna 'grupos':\n", df['grupo'].unique())


Valores únicos na coluna 'grupos':
 ['PÚBLICO EM GERAL (18 a 59 anos)' 'TRABALHADORES DA EDUCAÇÃO'
 'TRABALHADORES DA SAÚDE' 'IDOSOS' 'TRABALHADORES PORTUÁRIOS'
 'OUTRAS PRIORIDADES' 'PESSOAS COM COMORBIDADES' 'GESTANTES E PUÉRPERAS'
 'TRABALHADORES DE TRANSPORTE METROVIÁRIO E FERROVIÁRIO'
 'CRIANÇAS DE 05 A 11 ANOS' 'TRABALHADORES INDUSTRIAIS E BANCÁRIOS'
 'TRABALHADORES DA LIMPEZA URBANA'
 'PESSOAS COM VIAGEM PARA EXTERIOR (ESTUDO/PESQUISA/TRABALHO/TRATAMENTO DE SAÚDE)'
 'TRABALHADORES DE TRANSPORTE AÉREO' 'CAMINHONEIROS'
 'PESSOAS EM SITUAÇÃO DE RUA' 'PÚBLICO EM GERAL - MENOR DE 18 ANOS'
 'TRABALHADORES DE TRANSPORTE COLETIVO RODOVIÁRIO'
 'TRABALHADORES DA ASSISTÊNCIA SOCIAL'
 'TRABALHADORES DE TRANSPORTE AQUAVIÁRIO'
 'CRIANÇAS DE 05 A 11 ANOS - Prioritárias'
 'GESTANTES E PUÉRPERAS - 12 A 17 ANOS'
 'GESTANTES E PUÉRPERAS NÃO RESIDENTES EM RECIFE'
 'PESSOAS COM COMORBIDADES - 12 A 17 ANOS'
 'PESSOAS COM DEFICIÊNCIA - 12 A 17 ANOS'
 'CRIANÇAS DE 05 A 11 ANOS - Síndrome de Down' 'CRI

<p>Padronizando os valores:</P>

In [123]:
def simplificar_grupos(grupo_original):

    grupo = str(grupo_original)

    if 'CRIANÇAS' in grupo:
        return 'CRIANÇAS'
    
    elif'GESTANTES' in grupo or "PUÉRPERAS" in grupo:
        return "GESTANTES E PUÉRPERAS"
    
    elif "TRABALHADORES" in grupo or "CAMINHONEIROS" in grupo:
        return "TRABALHADORES (DIVERSOS)"
    
    elif "DEFICIÊNCIA" in grupo:
        return "PESSOAS COM DEFICIÊNCIA"
    
    elif "COMORBIDADES" in grupo:
        return "PESSOAS COM COMORBIDADES"
    
    elif "PÚBLICO" in grupo:
        return "PÚBLICO EM GERAL"
    
    elif "IDOSOS" in grupo:
        return "IDOSOS"
    
    elif "RUA" in grupo:
        return "PESSOAS EM SITUAÇÃO DE RUA"
    
    elif "VIAGEM" in grupo:
        return "PESSOAS COM VIAGEM PARA EXTERIOR"
    
    else:
        return "OUTRAS PRIORIDADES"
    
df['grupo'] = df['grupo'].apply(simplificar_grupos)

<p>Agora, vamos tratar a coluna "cnes"</p>

In [124]:
print("\nValores únicos na coluna 'cnes':\n", df['cnes'].unique())


Valores únicos na coluna 'cnes':
 ['DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER '
 'DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER'
 'DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BARROS'
 'DS 8: CNES: 0001198 - UBT ARISTARCHO DE AZEVEDO'
 'DS 6: CNES: 0001392 - MIGUEL DE LIMA VALVERDE.'
 'DS 3: CNES: 000612 - POLICLÍNICA ALBERT SABIN'
 'DS 5: CNES: 000701 - MATERNIDADE BANDEIRA FILHO'
 'DS 7: CNES: 0000647 - POLICLINICA CLEMENTINO FRAGA'
 'DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE']


<p>Nessa coluna, temos alguns problemas:</p>
<ul>
    <li>Existem dois valores que são o mesmo, mudando apenas que um deles, possuii um espaço no final</li>
    <li>Há valor com ponto final, mas a maioria, não está</li>
    <li>Cada valor é uma mistura de três valores: Distrito, Código e Nome da unidade. Podemos destrinchar e colocar cada um em colunas separadas, sendo elas: distrito_sanitario, codigo_cnes e nome_unidade</li>
</ul>

<p>Primeiro, vamos remover os espaços em branco e o ponto final:</p>

In [125]:
df['cnes'] = df['cnes'].astype(str).str.strip() #Removendo o espaço final em branco
df['cnes'] = df['cnes'].astype(str).str.rstrip('.') #Removendo o ponto final do final
print("\nValores únicos na coluna 'cnes':\n", df['cnes'].unique()) #Printando para confirmar


Valores únicos na coluna 'cnes':
 ['DS 2: CNES: 6897029 - POLICLÍNICA SALOMÃO KELNER'
 'DS 1: CNES: 000507 - POLICLÍNICA GOUVEIA DE BARROS'
 'DS 8: CNES: 0001198 - UBT ARISTARCHO DE AZEVEDO'
 'DS 6: CNES: 0001392 - MIGUEL DE LIMA VALVERDE'
 'DS 3: CNES: 000612 - POLICLÍNICA ALBERT SABIN'
 'DS 5: CNES: 000701 - MATERNIDADE BANDEIRA FILHO'
 'DS 7: CNES: 0000647 - POLICLINICA CLEMENTINO FRAGA'
 'DS 4: CNES: 0000639 - UBS JOAQUIM CAVALCANTE']


<p>Agora, vamos separar os dados para inserir nas colunas "distrito_sanitario", "codigo_cnes" e "nome_unidade"</p>

In [126]:
df_temp = df['cnes'].str.split(' - ', n=1, expand=True) #Separando a string e criando um dataframe temporário

df['codigo_cnes'] = df_temp[0] #Atribuindo a coluna 'codigo_cnes' a coluna 0 do df_temp
df['nome_unidade'] = df_temp[1].str.strip() #Atribuindo a coluna 'nome_unidade' a coluna 1 do df_temp

df['distrito_sanitario'] = df['codigo_cnes'].str.split(':', n=1).str[0].str.strip()
df['codigo_cnes'] = df['codigo_cnes'].str.split(':').str[2].str.strip()

df = df.drop('cnes', axis=1)

In [127]:
print(df.columns.tolist()) #Printando as colunas atuais só para confirmar
print("\nValores únicos na coluna 'cnes':\n", df['codigo_cnes'].unique()) #Printando para confirmar
print("\nValores únicos na coluna 'cnes':\n", df['nome_unidade'].unique()) #Printando para confirmar
print("\nValores únicos na coluna 'cnes':\n", df['distrito_sanitario'].unique()) #Printando para confirmar

['_id', 'faixa_etaria', 'idade', 'sexo', 'raca_cor', 'municipio', 'grupo', 'categoria', 'lote', 'vacina_fabricante', 'descricao_dose', 'sistema_origem', 'data_vacinacao', 'ano_origem', 'codigo_cnes', 'nome_unidade', 'distrito_sanitario']

Valores únicos na coluna 'cnes':
 ['6897029' '000507' '0001198' '0001392' '000612' '000701' '0000647'
 '0000639']

Valores únicos na coluna 'cnes':
 ['POLICLÍNICA SALOMÃO KELNER' 'POLICLÍNICA GOUVEIA DE BARROS'
 'UBT ARISTARCHO DE AZEVEDO' 'MIGUEL DE LIMA VALVERDE'
 'POLICLÍNICA ALBERT SABIN' 'MATERNIDADE BANDEIRA FILHO'
 'POLICLINICA CLEMENTINO FRAGA' 'UBS JOAQUIM CAVALCANTE']

Valores únicos na coluna 'cnes':
 ['DS 2' 'DS 1' 'DS 8' 'DS 6' 'DS 3' 'DS 5' 'DS 7' 'DS 4']


<p>Agora, vamos tratar a coluna "vacina_fabricante"</p>

In [128]:
print("\nValores únicos na coluna 'vacina_fabricante':\n", df['vacina_fabricante'].unique()) #Printando para confirmar


Valores únicos na coluna 'vacina_fabricante':
 ['3 - COMIRNATY (PFIZER)'
 '4 - JANSSEN COVID-19 VACCINE (JOHNSON & JOHNSON)'
 '2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ)'
 '1 - CORONAVAC - SINOVAC (BUTANTAN)'
 '6 - CORONAVAC - SINOVAC (BUTANTAN) PEDIÁTRICA'
 '5 - COMIRNATY (PFIZER) - PEDIÁTRICA']


<p>Assim como na coluna de "grupo", nessa também temos várias informações em uma string só, sendo elas:</p>
<ul>
    <li>1. ID da vacina</li>
    <li>2. Nome da vacina</li>
    <li>3. Fabricante/Laboratório</li>
    <li>4. Público </li>
</ul>

<p>Aplicaremos o mesmo tratamento que foi feito na coluna anterior. Iremos separar toda a string em 4 colunas, de acordo com a informação que representam.</p>

In [129]:
df['id_vacina'] = df['vacina_fabricante'].str.split('-').str[0].str.strip() #Primeiro, criando a coluna 'id_vacina" para amrzenar os IDs

df['fabricante_laboratorio'] = df['vacina_fabricante'].str.extract(r'\((.*?)\)') #Pegando o nome do laboratório que está entre parenteses

df['publico_alvo'] = df['vacina_fabricante'].str.split(') - ', n=1, regex=False).str[1] #Pegando o público alvo "PEDIATRA"
df['publico_alvo'] = df['publico_alvo'].fillna('GERAL') #Aplicando o fillna para que os valores nulos sejam preenchidos com "GERAL"

temp_nome = df['vacina_fabricante'].str.split(' - ', n=1).str[1] #Salvando uma parte temporaria da string
df['nome_vacina'] = temp_nome.str.split(' (', n=1, regex=False).str[0] #Aplicando split novamente para pegar o nome de fato

colunas_para_ver = ['id_vacina', 'fabricante_laboratorio', 'publico_alvo', 'nome_vacina']
print(display(df[colunas_para_ver].head()))

,id_vacina,fabricante_laboratorio,publico_alvo,nome_vacina
0,3,PFIZER,GERAL,COMIRNATY
1,3,PFIZER,GERAL,COMIRNATY
2,3,PFIZER,GERAL,COMIRNATY
3,4,JOHNSON & JOHNSON,GERAL,JANSSEN COVID-19 VACCINE
4,2,FIOCRUZ,GERAL,CHADOX1NCOV-19 - OXFORD/ASTRAZENECA


None


<p>Agora, iremos patrodinzar a coluna 'faixa_etaria'.</p>

In [130]:
df['faixa_etaria'].unique()

array(['35 a 39 anos', '50 a 54 anos', '55 a 59 anos', '40 a 44 anos',
       '30 a 34 anos', '75 a 79 anos', '25 a 29 anos', '20 a 24 anos',
       '60 a 64 anos', '65 a 69 anos', '70 a 74 anos', '10 a 14 anos',
       '45 a 49 anos', '80 a 84 anos', '15 a 19 anos', '5 a 9 anos',
       '90 a 94 anos', '85 a 89 anos', '95 a 99 anos', '100 anos +', nan,
       '0 a 4 anos'], dtype=object)

<h3>Removendo colunas</h3>

<p>Agora, vamos remover as seguintes colunas:</p>
<ul>
    <li>lote: A coluna 'lote' possui algunas valores nulos a serem tratados. Entretanto, acredito que ela em si não agregue muito para o dataset, então, iremos remove-la</li>
    <li>faixa_etaria: Apesar de estar bem padronizada, por já termos a idade, acredito que não precisamos dessa coluna.</li>
    <li>_id: A coluna "id" possui menos valores únicos do que a quantidade total de linhas, o que é um problema. Iremos remvoer a coluna e criar um novo id.</li>
</ul>

In [131]:
df = df.drop(columns=['_id', 'lote', 'faixa_etaria'], axis=1) #Removendo as colunas

df = df.reset_index(drop=True) #Removendo od indices antigos
df = df.reset_index() #Criando uma nova coluna "index" com novos indices
df = df.rename(columns={'index': 'id'}) #Renomeando o nome que foi criado pelo pandas para colocar "id"

display(df.head())


,id,idade,sexo,raca_cor,municipio,grupo,categoria,vacina_fabricante,descricao_dose,sistema_origem,data_vacinacao,ano_origem,codigo_cnes,nome_unidade,distrito_sanitario,id_vacina,fabricante_laboratorio,publico_alvo,nome_vacina
0,0,37.0,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),2,Conecta Recife,2022-03-25T00:00:00,2022,6897029,POLICLÍNICA SALOMÃO KELNER,DS 2,3,PFIZER,GERAL,COMIRNATY
1,1,52.0,MASCULINO,BRANCA,RECIFE,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),4,Conecta Recife,2022-12-06T00:00:00,2022,6897029,POLICLÍNICA SALOMÃO KELNER,DS 2,3,PFIZER,GERAL,COMIRNATY
2,2,58.0,FEMININO,BRANCA,OLINDA,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),1,Conecta Recife,2022-06-04T00:00:00,2022,000507,POLICLÍNICA GOUVEIA DE BARROS,DS 1,3,PFIZER,GERAL,COMIRNATY
3,3,35.0,MASCULINO,PARDA,RECIFE,TRABALHADORES (DIVERSOS),REDE PRIVADA - ENSINO FUNDAMENTAL,4 - JANSSEN COVID-19 VACCINE (JOHNSON & JOHNSON),3,Conecta Recife,2022-03-03T00:00:00,2022,0001198,UBT ARISTARCHO DE AZEVEDO,DS 8,4,JOHNSON & JOHNSON,GERAL,JANSSEN COVID-19 VACCINE
4,4,40.0,MASCULINO,PRETA,RECIFE,TRABALHADORES (DIVERSOS),OUTROS,2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ),2,Conecta Recife,2022-02-08T00:00:00,2022,0001392,MIGUEL DE LIMA VALVERDE,DS 6,2,FIOCRUZ,GERAL,CHADOX1NCOV-19 - OXFORD/ASTRAZENECA


<h3>Removendo valores nulos</h3>

<p>Após o tratamento dos valores nulos, ainda assim, sobraram duas colunas com valores nulos:</p>

In [132]:
print("Valores ausentes por coluna:")
print(df.isnull().sum())

Valores ausentes por coluna:
id                        0
idade                     1
sexo                      0
raca_cor                  0
municipio                 2
grupo                     0
categoria                 0
vacina_fabricante         0
descricao_dose            0
sistema_origem            0
data_vacinacao            0
ano_origem                0
codigo_cnes               0
nome_unidade              0
distrito_sanitario        0
id_vacina                 0
fabricante_laboratorio    0
publico_alvo              0
nome_vacina               0
dtype: int64


<p>Como são poucas linhas, irei remover as linhas com valores nulos.</p>

In [133]:
df = df.dropna()
print("Valores ausentes por coluna:")
print(df.isnull().sum())

Valores ausentes por coluna:
id                        0
idade                     0
sexo                      0
raca_cor                  0
municipio                 0
grupo                     0
categoria                 0
vacina_fabricante         0
descricao_dose            0
sistema_origem            0
data_vacinacao            0
ano_origem                0
codigo_cnes               0
nome_unidade              0
distrito_sanitario        0
id_vacina                 0
fabricante_laboratorio    0
publico_alvo              0
nome_vacina               0
dtype: int64


<h3>Alterando os tipos de algumas colunas</h3>

<p>Agora, iremos mudar o tipo das colunas 'idade' e 'ano_origem', que atualmente, estão como 'object'. Iremos alterar o tipo de ambas para inteiro.</p>

In [134]:
df['idade'] = df['idade'].astype(int)
df['ano_origem'] = df['ano_origem'].astype(int)

<p>Além disso, a coluna de "data_vacinacao" está cno formato estado unidense, então, iremos transforma-la para ficar no formato brasileiro.</p>

In [135]:
df['data_vacinacao'] =  pd.to_datetime(df['data_vacinacao']).dt.strftime('%d/%m/%Y')

<h3>Resultado final</h3>

<p>Terminamos a etapa de transformação. Vamos verificar como o nosso dataset ficou no final.</p>

In [136]:
display(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 291619 entries, 0 to 291621
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   id                      291619 non-null  int64 
 1   idade                   291619 non-null  int64 
 2   sexo                    291619 non-null  object
 3   raca_cor                291619 non-null  object
 4   municipio               291619 non-null  object
 5   grupo                   291619 non-null  object
 6   categoria               291619 non-null  object
 7   vacina_fabricante       291619 non-null  object
 8   descricao_dose          291619 non-null  int64 
 9   sistema_origem          291619 non-null  object
 10  data_vacinacao          291619 non-null  object
 11  ano_origem              291619 non-null  int64 
 12  codigo_cnes             291619 non-null  object
 13  nome_unidade            291619 non-null  object
 14  distrito_sanitario      291619 non-null  

None

,id,idade,sexo,raca_cor,municipio,grupo,categoria,vacina_fabricante,descricao_dose,sistema_origem,data_vacinacao,ano_origem,codigo_cnes,nome_unidade,distrito_sanitario,id_vacina,fabricante_laboratorio,publico_alvo,nome_vacina
0,0,37,MASCULINO,PARDA,RECIFE,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),2,Conecta Recife,25/03/2022,2022,6897029,POLICLÍNICA SALOMÃO KELNER,DS 2,3,PFIZER,GERAL,COMIRNATY
1,1,52,MASCULINO,BRANCA,RECIFE,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),4,Conecta Recife,06/12/2022,2022,6897029,POLICLÍNICA SALOMÃO KELNER,DS 2,3,PFIZER,GERAL,COMIRNATY
2,2,58,FEMININO,BRANCA,OLINDA,PÚBLICO EM GERAL,OUTROS,3 - COMIRNATY (PFIZER),1,Conecta Recife,04/06/2022,2022,000507,POLICLÍNICA GOUVEIA DE BARROS,DS 1,3,PFIZER,GERAL,COMIRNATY
3,3,35,MASCULINO,PARDA,RECIFE,TRABALHADORES (DIVERSOS),REDE PRIVADA - ENSINO FUNDAMENTAL,4 - JANSSEN COVID-19 VACCINE (JOHNSON & JOHNSON),3,Conecta Recife,03/03/2022,2022,0001198,UBT ARISTARCHO DE AZEVEDO,DS 8,4,JOHNSON & JOHNSON,GERAL,JANSSEN COVID-19 VACCINE
4,4,40,MASCULINO,PRETA,RECIFE,TRABALHADORES (DIVERSOS),OUTROS,2 - CHADOX1NCOV-19 - OXFORD/ASTRAZENECA (FIOCRUZ),2,Conecta Recife,08/02/2022,2022,0001392,MIGUEL DE LIMA VALVERDE,DS 6,2,FIOCRUZ,GERAL,CHADOX1NCOV-19 - OXFORD/ASTRAZENECA


<h2>Ccarregamento: Injetando Dataset no Banco de Dados PostgreSQL</h2>

In [137]:
#Instalando as dependências necessárias
!pip install sqlalchemy
!pip install psycopg2-binary

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   --------- ------------------------------ 0.5/2.1 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 7.9 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from sqlalchemy import create_engine

# Configurando a conexão
usuario_db = 'user'                  # Usuário padrão
senha_db = 'password'                # Senha
host_db = 'host'                     # Padrão
porta_db = 'port'                    # Padrão
nome_banco_db = 'database'           # O banco criado

# Criando a "connection string"
DATABASE_URL = f'postgresql://{usuario_db}:{senha_db}@{host_db}:{porta_db}/{nome_banco_db}'

print("Tentando criar conexão com o banco...")

try:
    engine = create_engine(DATABASE_URL)

    print("Conexão bem-sucedida! Iniciando a carga dos dados...")

    df.to_sql(
        'vacinados_etl_final',    # Nome da tabela que será criada
        con=engine,               # A conexão
        if_exists='replace',      # Apaga a tabela se ela já existir e cria de novo
        index=False               # NÃO salva o índice do Pandas
    )

    print(f"CARGA CONCLUÍDA! {len(df)} linhas carregadas na tabela 'vacinados_etl_final'.")

except Exception as e:
    print(f"\n--- ERRO! ---")
    print(f"Erro ao conectar ou carregar os dados: {e}")
    print("Verifique se o usuário, senha e nome do banco estão corretos.")

Tentando criar conexão com o banco...
Conexão bem-sucedida! Iniciando a carga dos dados...
CARGA CONCLUÍDA! 291619 linhas carregadas na tabela 'vacinados_etl_final'.
